In [2]:
import requests
from bs4 import BeautifulSoup

# URL principal
base_url = "http://books.toscrape.com/"

# Obtener contenido HTML
response = requests.get(base_url)
soup = BeautifulSoup(response.text, "html.parser")

# Buscar las categorías en el menú lateral
category_section = soup.find("ul", class_="nav-list").find("ul")
categories = category_section.find_all("a")

# Guardar las URLs completas de cada categoría
category_urls = []

for cat in categories:
    href = cat.get("href")
    full_url = base_url + href
    category_urls.append(full_url)

category_urls


['http://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'http://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'http://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'http://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'http://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'http://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'http://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'http://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'http://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'http://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'http://books.toscrape.com/catalogue/category/books/religion_12/index.html',
 'http://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html',
 'http://books.toscrape.com/catalogue/category/boo

In [ ]:

import requests
from bs4 import BeautifulSoup

all_book_urls = set()   # set para evitar duplicados

for category_url in category_urls:
    next_page = category_url

    while next_page:
        response = requests.get(next_page)
        soup = BeautifulSoup(response.text, "html.parser")

        # Obtener links de libros en la página actual
        books = soup.find_all("article", class_="product_pod")
        for book in books:
            href = book.find("h3").find("a").get("href")
            # Limpiar URL relativa
            href = href.replace("../../../", "")
            full_book_url = "http://books.toscrape.com/catalogue/" + href
            all_book_urls.add(full_book_url)

        # Verificar si existe un botón "Next" para paginación
        next_button = soup.find("li", class_="next")
        if next_button:
            next_link = next_button.find("a").get("href")
            # Construir siguiente página
            base = category_url.rsplit("/", 1)[0]  # cortar index.html
            next_page = base + "/" + next_link
        else:
            next_page = None

len(all_book_urls) 

print(list(all_book_urls)[:5])

print("Total de libros encontrados:", len(all_book_urls))




['http://books.toscrape.com/catalogue/we-the-people-the-modern-day-figures-who-have-reshaped-and-affirmed-the-founding-fathers-vision-of-america_302/index.html', 'http://books.toscrape.com/catalogue/the-third-wave-an-entrepreneurs-vision-of-the-future_862/index.html', 'http://books.toscrape.com/catalogue/foundation-foundation-publication-order-1_375/index.html', 'http://books.toscrape.com/catalogue/the-past-never-ends_942/index.html', 'http://books.toscrape.com/catalogue/el-deafo_691/index.html']
Total de libros encontrados: 1000


In [4]:
import sqlite3

conn = sqlite3.connect("Books.db")
cursor = conn.cursor()


In [5]:
# Crear tabla de libros
cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    price REAL,
    rating INTEGER,
    availability TEXT,
    category TEXT,
    description TEXT,
    upc TEXT UNIQUE
)
""")

# Crear tabla de autores
cursor.execute("""
CREATE TABLE IF NOT EXISTS authors (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

# Tabla intermedia libro <-> autor
cursor.execute("""
CREATE TABLE IF NOT EXISTS book_author (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_id INTEGER,
    author_id INTEGER,
    FOREIGN KEY (book_id) REFERENCES books(id),
    FOREIGN KEY (author_id) REFERENCES authors(id)
)
""")

conn.commit()

print("Base de datos y tablas creadas correctamente ✔️")


Base de datos y tablas creadas correctamente ✔️


In [6]:
import requests
from bs4 import BeautifulSoup


def scrape_book(url):
    import re
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Título
    title = soup.find("h1").text.strip()
    
    # Precio

    price_text = soup.find("p", class_="price_color").text.strip()
    price_clean = re.search(r'[\d.]+', price_text).group()
    price = float(price_clean)

    
    # Rating
    rating_dict = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
    rating_class = soup.find("p", class_="star-rating")["class"]
    rating_text = [r for r in rating_class if r in rating_dict][0]
    rating = rating_dict[rating_text]
    
    # Disponibilidad
    availability_text = soup.find("p", class_="instock availability").text.strip()
    import re
    stock_match = re.search(r'(\d+)', availability_text)
    stock = int(stock_match.group(1)) if stock_match else 0
    
    # Categoría
    category = soup.find("ul", class_="breadcrumb").find_all("a")[2].text.strip()
    
    # Descripción
    desc_tag = soup.find("div", id="product_description")
    if desc_tag:
        description = desc_tag.find_next_sibling("p").text.strip()
    else:
        description = ""
    
    # UPC
    upc = soup.find("th", text="UPC").find_next_sibling("td").text.strip()
    
    # Autor (si no está, poner "Desconocido")
    author = soup.find("table", class_="table table-striped").find("th", text="Product Type")
    author = "Desconocido"  # Books To Scrape no tiene campo explícito para autor, se puede reemplazar según convenga
    
    return {
        "title": title,
        "price": price,
        "rating": rating,
        "availability": availability_text,
        "category": category,
        "description": description,
        "upc": upc,
        "author": author
    }

# Prueba rápida
example_url = list(all_book_urls)[0]
book_data = scrape_book(example_url)
book_data


C:\Users\Hp i3\AppData\Local\Temp\ipykernel_5996\2201204750.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = soup.find("th", text="UPC").find_next_sibling("td").text.strip()
C:\Users\Hp i3\AppData\Local\Temp\ipykernel_5996\2201204750.py:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author = soup.find("table", class_="table table-striped").find("th", text="Product Type")


{'title': "We the People: The Modern-Day Figures Who Have Reshaped and Affirmed the Founding Fathers' Vision of America",
 'price': 31.95,
 'rating': 3,
 'availability': 'In stock (3 available)',
 'category': 'Nonfiction',
 'description': 'What would the Founding Fathers think about America today? Over 200 years ago the Founders broke away from the tyranny of the British Empire to build a nation based on the principles of freedom, equal rights, and opportunity for all men. But life in the United States today is vastly different from anything the original Founders could have imagined in the late 1700s. The no What would the Founding Fathers think about America today? Over 200 years ago the Founders broke away from the tyranny of the British Empire to build a nation based on the principles of freedom, equal rights, and opportunity for all men. But life in the United States today is vastly different from anything the original Founders could have imagined in the late 1700s. The notion of a

In [ ]:
import sqlite3
import time

# Conectar a la base de datos
conn = sqlite3.connect("books.db")
cursor = conn.cursor()

# Configuración de reintentos
max_retries = 5
delay_between_requests = 0.3  # segundos

# Recorrer todas las URLs de libros
total = len(all_book_urls)
for i, url in enumerate(all_book_urls, start=1):
    try:
        book = scrape_book(url)
        
        # Insertar autor con reintentos
        for attempt in range(max_retries):
            try:
                cursor.execute("SELECT id FROM authors WHERE name = ?", (book['author'],))
                author_row = cursor.fetchone()
                if author_row:
                    author_id = author_row[0]
                else:
                    cursor.execute("INSERT INTO authors (name) VALUES (?)", (book['author'],))
                    author_id = cursor.lastrowid
                conn.commit()
                break 
            except sqlite3.OperationalError as e:
                if "locked" in str(e):
                    print("DB locked al insertar autor, reintentando...")
                    time.sleep(0.5)
                else:
                    raise
        
        # Insertar libro con reintentos
        for attempt in range(max_retries):
            try:
                cursor.execute("""
                    INSERT OR IGNORE INTO books (title, price, rating, availability, category, description, upc)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                """, (
                    book['title'],
                    book['price'],
                    book['rating'],
                    book['availability'],
                    book['category'],
                    book['description'],
                    book['upc']
                ))
                conn.commit()
                break
            except sqlite3.OperationalError as e:
                if "locked" in str(e):
                    print("DB locked al insertar libro, reintentando...")
                    time.sleep(0.5)
                else: 
                    raise
        
        # Obtener book_id con reintentos
        for attempt in range(max_retries):
            try:
                cursor.execute("SELECT id FROM books WHERE upc = ?", (book['upc'],))
                book_id = cursor.fetchone()[0]
                break
            except sqlite3.OperationalError as e:
                if "locked" in str(e):
                    print("DB locked al obtener book_id, reintentando...")
                    time.sleep(0.5)
                else:
                    raise
        
        # Insertar relación libro ↔ autor con reintentos
        for attempt in range(max_retries):
            try:
                cursor.execute("""
                    INSERT OR IGNORE INTO book_author (book_id, author_id)
                    VALUES (?, ?)
                """, (book_id, author_id))
                conn.commit()
                break
            except sqlite3.OperationalError as e:
                if "locked" in str(e):
                    print("DB locked al insertar relación, reintentando...")
                    time.sleep(0.5)
                else:
                    raise
        
        # Mostrar progreso cada 50 libros
        if i % 50 == 0 or i == total:
            print(f"Procesados {i}/{total} libros")
    
    except Exception as e:
        print(f"Error en {url}: {e}")
    
    # Pausa entre requests
    time.sleep(delay_between_requests)

# Cerrar conexión
conn.close()
print("Todos los libros guardados en la base de datos ✔️")


C:\Users\Hp i3\AppData\Local\Temp\ipykernel_5996\2201204750.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc = soup.find("th", text="UPC").find_next_sibling("td").text.strip()
C:\Users\Hp i3\AppData\Local\Temp\ipykernel_5996\2201204750.py:46: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  author = soup.find("table", class_="table table-striped").find("th", text="Product Type")


Procesados 50/1000 libros
Procesados 100/1000 libros
Procesados 150/1000 libros
Procesados 200/1000 libros
Procesados 250/1000 libros
Procesados 300/1000 libros
Procesados 350/1000 libros
Procesados 400/1000 libros
Procesados 450/1000 libros
Procesados 500/1000 libros
Procesados 550/1000 libros
Procesados 600/1000 libros
Procesados 650/1000 libros
Procesados 700/1000 libros
Procesados 750/1000 libros
Procesados 800/1000 libros
Procesados 850/1000 libros
Procesados 900/1000 libros
Procesados 950/1000 libros
Procesados 1000/1000 libros
Todos los libros guardados en la base de datos ✔️


In [ ]:
import sqlite3
import pandas as pd  

df_books_authors = pd.read_sql_query("""
SELECT b.title, a.name AS author
FROM books b
JOIN book_author ba ON b.id = ba.book_id
JOIN authors a ON a.id = ba.author_id;
""", conn)

df_top_expensive = pd.read_sql_query("""
SELECT title, price
FROM books
ORDER BY price DESC
LIMIT 5;
""", conn)

df_category_count = pd.read_sql_query("""
SELECT category, COUNT(*) AS total_books
FROM books
GROUP BY category
ORDER BY total_books DESC;
""", conn)

df_authors_rank = pd.read_sql_query("""
SELECT a.name, COUNT(ba.book_id) AS total_books
FROM authors a
JOIN book_author ba ON a.id = ba.author_id
GROUP BY a.name
ORDER BY total_books DESC;
""", conn)


In [ ]:
+------------------+
|      Author      |
+------------------+
| id (PK)          |
| name             |
+------------------+

+------------------+
|      Book        |
+------------------+
| id (PK)          |
| title            |
| price            |
| rating           |
| availability     |
| category         |
| description      |
| upc (UNIQUE)     |
+------------------+

+----------------------+
|     Book_Author      |
+----------------------+
| book_id (FK)         |
| author_id (FK)       |
+----------------------+


In [ ]:
Usuario
   |
   v
Script Python
   |
   v
Scrape Book URL
   |
   v
Extraer datos (title, price, etc)
   |
   v
Buscar autor en DB
   |
   |-- Existe? ----> Sí ----> Obtener author_id
   |                   |
   |                   No
   |                   |
   |              Insertar autor
   |
   v
Insertar libro
   |
   v
Obtener book_id
   |
   v
Insertar relación book_author
   |
   v
Commit
